# Generate Summary


This step generates a json file compiling all the output from the previous steps.

It then uses a jinja2 template to parse that json file into an HTML page. This page is the summary and can be manually edited to add or remove information.

As this requires neither human intervention, or clinical information about the n-of-1 sample, many fields in the summary document will be blank and require filling in before deliverable to downstream viewers.


In [ ]:
import os
import json
import errno
from jinja2 import Environment, FileSystemLoader 
import logging

# Setup: load conf, retrieve sample ID, logging
with open("conf.json","r") as conf:
    c=json.load(conf)
sample_id = c["sample_id"]    
print("Running on sample: {}".format(sample_id))

logging.basicConfig(**c["info"]["logging_config"])
logging.info("\n9: Generate Summary Document")
def and_log(s):
    logging.info(s)
    return s

In [ ]:
# Get all the json -- outputs, conf, and analysis failed info
all_json = {}
for notebook_num, path in c["json"].iteritems():  
    newname = "nb_{}".format(notebook_num.replace(".", "pt"))
    with open(path) as this_jsonfile:
        all_json[newname] = json.load(this_jsonfile)
        

all_json["conf"] = c
all_json["sample_id"] = sample_id

try:
    with open(c["file"]["flag_analysis_failed"], "r") as jf:
        all_json["analysis_failed"] = json.load(jf)
except IOError, e:
    if e.errno == errno.ENOENT:
        all_json["analysis_failed"] = False
    else:
        raise

In [ ]:
# Set up the template engine and write the output files
# Always escape content
env=Environment(autoescape=True,
                loader=FileSystemLoader(".")
               )

# For embedding any files (eg css) raw, without parsing contents
def get_resource_as_string(name, charset='utf-8'):
    with open(name, "r") as f:
        return f.read().decode(charset)

env.globals['get_resource_as_string'] = get_resource_as_string

findings_template=env.get_template(c["info"]["summary_template_name"])
slides_template = env.get_template(c["info"]["slides_template_name"])

# Write the final  summary HTML
print("Rendering findings to {}".format(c["file"]["summary_html"]))
with open(c["file"]["summary_html"], "w") as r:
    r.write(findings_template.render(all_json).encode('utf-8'))
    
print("Rendering slides to {}".format(c["file"]["slides_html"]))
with open(c["file"]["slides_html"], "w") as r:
    r.write(slides_template.render(all_json).encode('utf-8'))

In [ ]:
logging.info("Tertiary Analysis complete.")
print 'Done!'